# Demo workflow: *EqV2* H-adsorption relaxations on (2 2 1) slabs

This Jupyter notebook reproduces the key steps used in ACS journal submitted article.
It demonstrates how to:
1. set up the exact software environment (matching the paper),
2. build (2 2 1) slabs from a small Bulk dataset,
3. generate heuristic **+** random H adsorption sites,
4. relax each configuration with the *EquiformerV2-31 M* surrogate model, and
5. post-process trajectories to extract *min-energy* structures, force metrics, adsorption‐site type, and slab metadata.

The notebook is self-contained: simply run sequentially inside any **GPU‐enabled** Python 3.10+ environment.


## 0. Software setup

In [ ]:
!pip install --quiet --extra-index-url https://download.pytorch.org/whl/cu121 \
            torch==2.3.0+cu121 torchvision==0.18.0+cu121 torchaudio==2.3.0+cu121
!pip install --quiet pyg_lib torch_scatter==2.1.2+pt23cu121 torch_sparse==0.6.18+pt23cu121 \
            torch_cluster==1.6.3+pt23cu121 torch_spline_conv==1.2.2+pt23cu121 \
            -f https://data.pyg.org/whl/torch-2.3.0+cu121.html
!pip install --quiet torch_geometric==2.6.1
!pip install --quiet numpy==1.26.4 pandas==2.2.2 fairchem-core==1.2.0 fairchem-data-oc==0.0.1 \
            pymatgen==2024.10.3 ase==3.23.0 tqdm

## 1. Imports & input data

In [ ]:
import os, time, pickle, shutil, re
from pathlib import Path
import numpy as np, pandas as pd, ase.io as aseio
from tqdm import tqdm

from fairchem.core.common.relaxation.ase_utils import OCPCalculator
from fairchem.core.models.model_registry import model_name_to_local_file
from fairchem.data.oc.core import Bulk, Slab, Adsorbate, AdsorbateSlabConfig
from fairchem.data.oc.utils import DetectTrajAnomaly
from ase.optimize import BFGS
import fairchem.data.oc

# ---- load a small bulk dataset --------------------------------------------------
work_pkl = Path('InDomainBulk.pkl')              # <-- provide with repo (tiny demo set)
assert work_pkl.exists(), 'InDomainBulk.pkl not found!'  # keep users honest

with open(work_pkl, 'rb') as f:
    bulks = pickle.load(f)
bulk_ids = [b['src_id'] for b in bulks][:2]      # demo → first two IDs
print('Demo bulk IDs:', bulk_ids)

# adsorbate DB
ads_db = Path(fairchem.data.oc.__file__).parent/'databases/pkls/adsorbates.pkl'
ads_H  = Adsorbate(adsorbate_smiles_from_db='*H', adsorbate_db_path=ads_db)


## 2. Surrogate calculator

In [ ]:
ckpt = model_name_to_local_file('EquiformerV2-31M-S2EF-OC20-All+MD',
                               local_cache='/tmp/ocp_checkpoints/')
calc = OCPCalculator(checkpoint_path=ckpt, cpu=False)


## 3. Build (2 2 1) slabs & relax H configurations

In [ ]:
ORIENT      = (2, 2, 1)
orient_tag  = ''.join(map(str, ORIENT))          # '221'
OUT_ROOT    = f'InDomain{orient_tag}'
os.makedirs(OUT_ROOT, exist_ok=True)

start = time.time()

for bid in tqdm(bulk_ids, desc='Bulks'):
    bulk  = Bulk(bulk_src_id_from_db=bid, bulk_db_path=str(work_pkl))
    slabs = Slab.from_bulk_get_specific_millers(bulk=bulk, specific_millers=ORIENT)

    for sidx, slab in enumerate(slabs):
        heur = AdsorbateSlabConfig(slab, ads_H, mode='heuristic')
        rand = AdsorbateSlabConfig(slab, ads_H, mode='random_site_heuristic_placement', num_sites=20)
        configs = [*heur.atoms_list, *rand.atoms_list]

        out_dir = f'{OUT_ROOT}/{bid}_H_{sidx}'
        os.makedirs(out_dir, exist_ok=True)
        print(f'🟢 {len(configs)} configs → {out_dir}')

        for cidx, atoms in enumerate(configs):
            atoms.calc = calc
            BFGS(atoms,
                 trajectory=f'{out_dir}/config_{cidx}.traj',
                 logfile   =f'{out_dir}/config_{cidx}.log').run(fmax=0.05, steps=100)

print('Relaxations finished in', round(time.time()-start,1), 's')


## 4. Post-process trajectories → min-energy table

In [ ]:
records = []
for slab_dir in Path(OUT_ROOT).glob('*_H_*'):
    base    = slab_dir.name                  # e.g. mp-123_H_0
    bulk_id, _, slab_idx = base.split('_')   # slab_idx str
    key = f'{bulk_id}_H_{slab_idx}_{orient_tag}'

    good = []
    for traj_file in slab_dir.glob('*.traj'):
        traj = aseio.read(traj_file, ':')
        det  = DetectTrajAnomaly(traj[0], traj[-1], traj[0].get_tags())
        if any([det.is_adsorbate_dissociated(), det.is_adsorbate_desorbed(),
                det.has_surface_changed(), det.is_adsorbate_intercalated()]):
            continue
        E = traj[-1].get_potential_energy()
        F = traj[-1].get_forces()
        good.append({'E':E, 'F':F, 'file':str(traj_file)})

    if good:
        best  = min(good, key=lambda d: d['E'])
        max_F = np.linalg.norm(best['F'], axis=1).max()
        records.append(dict(key=key, bulk_id=bulk_id, slab_index=int(slab_idx),
                           orientation=orient_tag, adsorbate='H',
                           min_E_ml=best['E'], max_F_ml=max_F,
                           traj_file=best['file']))

df_min = pd.DataFrame(records)
print('Rows collected :', len(df_min))
df_min.head()


## 5. Adsorption-site classification

In [ ]:
import math
from ase.io import read

# ------------------------------------------------------------------
# helper 1 – tile slab until the two in-plane cell vectors ≥ min_ab Å
# ------------------------------------------------------------------
def tile_substrate(atoms, min_ab: float = 8.0):
    tags = atoms.get_tags()
    sub, ads = atoms[tags != 2], atoms[tags == 2]          # adsorbate has tag==2
    a_len, b_len = np.linalg.norm(sub.cell[0]), np.linalg.norm(sub.cell[1])
    na, nb = max(1, math.ceil(min_ab / a_len)), max(1, math.ceil(min_ab / b_len))
    return sub.repeat((na, nb, 1)) + ads

# simple map: neighbour-count → site label
classify = {1: "top", 2: "bridge", 3: "3-fold", 4: "4-fold"}.get

# ------------------------------------------------------------------
# helper 2 – indices of nearest surface atoms to adsorbate
# ------------------------------------------------------------------
def nearest_idx(atoms, ads_i: int, surf_idx, tol: float = 0.10):
    """Return indices of all surface atoms within (1+tol)*d_min of adsorbate."""
    dists = atoms.get_distances(ads_i, surf_idx, mic=True)
    pairs = sorted([(d, j) for d, j in zip(dists, surf_idx) if d > 0],
                   key=lambda x: x[0])

    if not pairs:                       # <-- safety: empty list
        return []

    cutoff = pairs[0][0] * (1 + tol)    # first entry = d_min
    return [j for d, j in pairs if d <= cutoff]

# ------------------------------------------------------------------
# main row-wise routine
# ------------------------------------------------------------------
def site_info(row):
    """Attach site type & nearest surface-atom symbols for one relaxed traj."""
    try:
        atoms = read(row.traj_file, index=-1)              # last frame only
        tiled = tile_substrate(atoms)                      # enlarge supercell
        tags  = tiled.get_tags()
        ads_i = int(np.where(tags == 2)[0][0])             # first adsorbate atom
        surf  = np.where(tags == 1)[0]                     # surface atoms
        near  = nearest_idx(tiled, ads_i, surf)

        site_type = classify(len(near), "unknown")
        near_syms = [tiled[i].symbol for i in near] if near else None

        return pd.Series({"site_type": site_type,
                          "nearest_atoms": near_syms})
    except Exception as err:
        # if anything goes wrong keep the DF intact & show None
        return pd.Series({"site_type": None,
                          "nearest_atoms": None})

# ------------------------------------------------------------------
# apply to your minima DataFrame
# ------------------------------------------------------------------
df_min[["site_type", "nearest_atoms"]] = df_min.apply(site_info, axis=1)
df_min.head()


## 6. Add slab shift / top metadata

In [ ]:
import numpy as np
def shift_top(slab):
    m = re.search(r'\([^)]*\),\s*([-+]?\d*\.?\d+),\s*(True|False)', str(slab))
    return (np.nan,np.nan) if m is None else (float(m.group(1)), m.group(2)=='True')

cache = {}
for key in df_min.key.unique():
    bid,_,idx,_ = key.split('_')
    idx = int(idx)
    if (bid,idx) in cache: continue
    slabs = Slab.from_bulk_get_specific_millers(Bulk(bid, str(work_pkl)), ORIENT)
    cache[(bid,idx)] = shift_top(slabs[idx]) if idx < len(slabs) else (np.nan,np.nan)

df_min[['shift','top']] = df_min.apply(lambda r: pd.Series(cache[(r.bulk_id,r.slab_index)]), axis=1)
df_min.head()


## 7. Save results

In [ ]:
out_csv = f'{OUT_ROOT}_minE_clean.csv'
df_min.to_csv(out_csv, index=False)
print('Saved →', out_csv)


---
**Notebook generated:** 2025-07-10 19:21 UTC

*Correspondence*: i.c.oguz@differ.nl